### GPT API로 50개씩.csv 데이터 돌리기

In [1]:
import pandas as pd
import re
import json
import time

from langchain_openai import ChatOpenAI
import openai
from dotenv import load_dotenv
import os

load_dotenv()

False

In [30]:
df = pd.read_csv("./카테고리 태깅 프롬프트 엔지니어링/GPT API 프롬프팅/카테고리_태깅된_문장.csv")
df

,product_id,product_type,review,category
0,5013198,한우 도가니탕,하트찜꽁 쫀득한 도가니탕 맛있네요,맛/향/풍미
1,5013198,한우 도가니탕,국 간은 싱거운데 소금 간 하면 딱이에요,맛/향/풍미
2,5013198,한우 도가니탕,가격은 음 착하지는 않습니다,가격
3,5013198,한우 도가니탕,기름 너무 많아서 느끼함,맛/향/풍미
4,5013198,한우 도가니탕,믿고 먹는 설성 목장이예요,맛/향/풍미
...,...,...,...,...
25471,1000356831,얼큰곤이알탕 400g,얼큰하고 맛도 있으나,맛/향/풍미
25472,1000356831,얼큰곤이알탕 400g,건데기 알과 곤이가 너무 적음,양
25473,1000356831,얼큰곤이알탕 400g,양이 적지만 한 끼 먹기는 좋아요,양
25474,1000356831,얼큰곤이알탕 400g,알이 작고 적게 들었어요,양


In [9]:
filtered_df = df[df['category'] == '활용 방법']#.drop_duplicates()
filtered_df['product_id'].value_counts()

1000175407    115
1000128834     61
1000347300     47
5144534        43
5050925        43
             ... 
1000134525      4
5057668         4
5013198         4
1000239335      3
5136026         2
Name: product_id, Length: 267, dtype: int64

In [27]:
combined = filtered_df.groupby('product_id')['review'].agg(' '.join).reset_index()
filtered_combined = combined[combined['review'].apply(lambda x: len(x.split()) >= 10)]
filtered_combined

,product_id,review
0,5001235,집에 있는 새 송이버섯 애호박 대파 넣어 주고 계란 하나 푸니까 애정템 입니다 파 ...
1,5005071,추후 구매 후 참치 3 캔 기름기 빼서 넣었더니 아이들이 잘 먹네요 두부나 찌개 채...
2,5011633,어른처럼 소면 좋아하는 아이는 삶아서 넣어 먹으면 곰국 국수 질 좋은 다른 곰국도 ...
3,5012851,리뷰가 좋아서 구매했어요 아침에 한 끼 국으로 좋았어요 청꼬 넣어서 푹 끓여서 한 ...
4,5012950,닭 계장할 때 베이스로 사골 국물 이용해서 쉽게 깊은 맛 냈어요 걍 찌개 베이스로 ...
...,...,...
262,1000538774,삼계탕은 요걸로 정착입니다 삼계탕 마니아로써 조심스럽게 말씀 드리는데 올 여름 집에...
263,1000605720,집에 있는 새우 더 넣고 쫄면 추가해서 짬뽕으로 먹었어요 다음엔 우동 면 추가해서 ...
264,1000605738,더워서 입맛이 없으시다면 소반 옥 육개장에 밥 말아서 한 그릇 뚝딱 해 보시는 걸 ...
265,1000620989,배송을 받으시면 냉동고에 넣지 마시고 냉장고에 넣어서 하루 이상 냉장고에서 완전히 ...


In [30]:
filtered_combined[filtered_combined['product_id']==5025668].loc[6, 'review']

'물 약간 넣어서 드렸어요 너무 좋아요 잘 먹고 있어요 간편하고 아욱 된장국 간편하게 헌 끼 먹기 좋아요 동전 육수 추가해서 먹었어요 두부 넣어서 먹으면 더 맛있어요 휘리릭 끓여 주면 잘 먹습니다 밥만 지어 맛있게 먹었어요 가끔 국 끓이기 싫을 때 주문해서 먹어요 냉동시켜서 데워 먹기도 편해요 두부 넣고 끓이면 맛있어요 집 된장과 물을 조금 더 넣고 양을 늘려 여러 번 먹습니다'

In [46]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=1000,
    openai_api_key="openai_api_key"
)

In [ ]:
example_recipe = """
'''
이렇게 활용하면 좋아요 : 만둣국, 갈비탕
추가하면 좋아요 : 물, 대파, 기름 적은 소고기, 밥
조리 방법 : 물을 먼저 넣고 팔팔 끓이다가 소고기를 넣어주세요. 마지막에 밥을 말아 드세요.
편리함 : 냄비에 한 번에 넣고 끓이면 끝
'''

'''
이렇게 활용하면 좋아요 : 해물 오뎅탕
추가하면 좋아요 : 해산물, 오뎅 사리, 무, 대파, 후추, 소금
'''

'''
추가하면 좋아요 : 샤브샤브소고기, 차돌박이, 칼국수면
조리 방법 : 다 먹고 칼국수면을 넣어서 끓여 드세요.
편리함 : 전자레인지에 돌리기만 해도 완성
'''

'''
이렇게 활용하면 좋아요 : 김치찌개, 라면
추가하면 좋아요 : 김치, 라면사리, 각종 야채
조리 방법 : 김치를 많이 넣고, 각종 야채는 조금만 추가해주세요.
'''
"""

In [47]:
prompt = f"""
지시사항:
아래 주어진 '문제'는 특정 상품의 리뷰에서 소비자들이 제시한 상품의 활용 방법에 대해서만 추출한 내용입니다.
새로운 소비자 이 상품을 어떻게 사용하면 좋을지 예시와 같이 적절히 요약해주세요.

예시:
"라면사리, 각종 야채를 추가하여 부대찌개로 먹으면 맛있습니다.
냄비에 한 번에 넣고 끓이면 돼서 간편합니다. 어른들 해장용으로도 좋고, 아이들 입맛에도 잘 맞습니다."

"송이버섯, 애호박, 대파 등 집에 있는 야채를 듬뿍 넣어 만둣국으로 끓여 먹기 좋습니다.
다 먹고 남은 국물에 칼국수면을 넣어서 끓여 드세요. 추운 겨울에 몸보신하기 딱입니다."

"전자레인지에 돌리기만 하면 손쉽게 든든한 한 끼를 먹을 수 있습니다.
자취생들이 간편하게 먹을 수 있습니다. 김치랑 같이 먹으면 잘 어울립니다."


제한사항:
브랜드 이름은 넣지 마세요.
'소비자', '리뷰', '요약'이라는 단어는 사용하지 마세요.

"""

In [48]:
def summarizing(text):
    prompting = (
    f"{prompt}\n"
    "**문제**\n"
    f"'''{text}'''\n" 
    )
    
    messages = [("system", "You are an analyst who provides summarization of reviews of products."),
                ("user", prompting)]
    
    response = llm.invoke(messages)     
    return response
    #return messages

In [31]:
summarized = []

In [32]:
for i in range(filtered_combined.shape[0]):
    print(i)
    text = filtered_combined.loc[i, 'review']
    response = summarizing(text)
    summarized.append(response.content.strip('"'))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266


In [33]:
summarized 

['집에 있는 새 송이버섯, 애호박, 대파를 넣고 계란을 풀어 간편하게 요리할 수 있습니다. 양 곰탕에 물을 추가하고 고춧가루와 마늘을 넣어 끓인 후, 당면과 계란을 추가하면 맛있는 한 끼가 완성됩니다. 아기에게도 적합하며, 순두부를 넣어 순두부찌개처럼 즐길 수도 있습니다. 또한, 부추와 버섯을 추가해 달걀을 풀어 먹으면 더욱 맛있습니다. 전골 육수로 활용할 때는 다양한 야채를 넣고 쪽파를 사용하면 아이들도 잘 먹습니다. 해장용으로도 좋고, 채소를 더 추가해 전골로 즐기기에도 적합합니다.',
 '참치 캔을 기름기 빼고 넣어 아이들과 함께 즐기기 좋습니다. 두부와 찌개 채소를 추가해 끓이면 든든한 한 끼가 됩니다. 물을 조금 더 넣고 마늘을 듬뿍 넣어 끓이면 더욱 맛있습니다. 갓 지은 밥과 함께 먹으면 최고의 조합입니다. 자취생들에게도 간편하고 맛있는 선택이 될 것입니다.',
 '소면을 삶아 곰국에 넣어 먹으면 맛있습니다. 아침에 밥을 말아 주기 좋고, 반찬이 없을 때 급하게 활용할 수 있습니다. 아기 국을 만들 때 기본 재료로 사용하거나, 국물 요리가 필요할 때 간편하게 데워 먹기 좋습니다. 주말 아침 떡국을 끓일 때 두 팩을 사용하면 더욱 풍성하게 즐길 수 있습니다. 만두나 떡을 넣고 끓여도 좋고, 비상용으로 쟁여 두면 유용합니다. 사골 육수를 베이스로 다양한 요리를 만들 수 있어 간편하고 좋습니다.',
 '아침에 간편하게 국으로 즐기기 좋습니다. 청꼬와 함께 푹 끓여서 한 끼 해결할 수 있으며, 밥 대신 국으로 간단히 먹기 좋습니다. 두부와 대파를 추가하면 더욱 맛있고, 계란을 풀어 넣으면 풍미가 더해집니다. 술 먹은 다음날 해장용으로도 적합하며, 국 끓이기 귀찮을 때 간편하게 주문해 먹기 좋습니다.',
 '사골 국물은 다양한 요리에 활용하기 좋습니다. 떡국이나 만두국을 만들 때 기본 육수로 사용하면 깊은 맛을 낼 수 있습니다. 간편하게 찌개를 끓일 때도 적합하며, 김치찌개나 부대찌개에 넣으면 풍미가 더해집니다. 또한, 국물 요리나 떡볶이, 닭 칼국수 등에

In [34]:
len(summarized)

267

In [37]:
filtered_combined['summarizing'] = summarized
filtered_combined

,product_id,review,summarizing
0,5001235,집에 있는 새 송이버섯 애호박 대파 넣어 주고 계란 하나 푸니까 애정템 입니다 파 ...,"집에 있는 새 송이버섯, 애호박, 대파를 넣고 계란을 풀어 간편하게 요리할 수 있습..."
1,5005071,추후 구매 후 참치 3 캔 기름기 빼서 넣었더니 아이들이 잘 먹네요 두부나 찌개 채...,참치 캔을 기름기 빼고 넣어 아이들과 함께 즐기기 좋습니다. 두부와 찌개 채소를 추...
2,5011633,어른처럼 소면 좋아하는 아이는 삶아서 넣어 먹으면 곰국 국수 질 좋은 다른 곰국도 ...,"소면을 삶아 곰국에 넣어 먹으면 맛있습니다. 아침에 밥을 말아 주기 좋고, 반찬이 ..."
3,5012851,리뷰가 좋아서 구매했어요 아침에 한 끼 국으로 좋았어요 청꼬 넣어서 푹 끓여서 한 ...,아침에 간편하게 국으로 즐기기 좋습니다. 청꼬와 함께 푹 끓여서 한 끼 해결할 수 ...
4,5012950,닭 계장할 때 베이스로 사골 국물 이용해서 쉽게 깊은 맛 냈어요 걍 찌개 베이스로 ...,사골 국물은 다양한 요리에 활용하기 좋습니다. 떡국이나 만두국을 만들 때 기본 육수...
...,...,...,...
262,1000538774,삼계탕은 요걸로 정착입니다 삼계탕 마니아로써 조심스럽게 말씀 드리는데 올 여름 집에...,삼계탕을 간편하게 데워서 여름철 보양식으로 즐기기 좋습니다. 조리 방법이 간단해 냉...
263,1000605720,집에 있는 새우 더 넣고 쫄면 추가해서 짬뽕으로 먹었어요 다음엔 우동 면 추가해서 ...,집에 있는 새우와 쫄면을 추가해 짬뽕으로 즐기면 좋습니다. 다음에는 우동 면을 넣어...
264,1000605738,더워서 입맛이 없으시다면 소반 옥 육개장에 밥 말아서 한 그릇 뚝딱 해 보시는 걸 ...,"더운 날 입맛이 없을 때, 얼린 육개장을 냄비에 넣고 끓여서 밥과 함께 먹으면 간편..."
265,1000620989,배송을 받으시면 냉동고에 넣지 마시고 냉장고에 넣어서 하루 이상 냉장고에서 완전히 ...,"냉장고에서 하루 이상 해동한 후, 냄비에 옮겨 물을 약 1312ml 정도 넣고 끓이..."


In [41]:
filtered_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267 entries, 0 to 266
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   product_id   267 non-null    int64 
 1   review       267 non-null    object
 2   summarizing  267 non-null    object
dtypes: int64(1), object(2)
memory usage: 16.4+ KB


In [50]:
filtered_combined.loc[262, 'review']

'삼계탕은 요걸로 정착입니다 삼계탕 마니아로써 조심스럽게 말씀 드리는데 올 여름 집에서 먹는 삼계탕으로 좋은 대안이 될 것 같습니당 초복 중복 다 요 제품으로 챙기면 될 것 같아요 간단하게 데워 먹으니 진짜 편해요 조리도 편해서 좋아요 몸 보신 하고 싶어서 간단하게 주문해 봤어요 집에서 편하게 먹고 싶어서 완제품으로 첫 도전인데 너무 만족스러웠어요 남은 국물로 닭 죽까지 완벽하게 먹었답니다 아이랑 함께 맛있게 먹었습니다 냉동된 제품을 팩 채 끓는 물에 넣어 놓고 기다리면 끝나는 간단한 조리 제품이네요 사서 냉장고에 하루 정도 해동 하고 중탕으로 끓여서 먹어 봤는데 요 정도면 초복 중복 다 맡겨도 되겠어요 부모님 댁에도 보내 드려야겠습니다 이번 여름은 녹두 삼계탕으로 이겨 내 볼게요 봉지 뜯어 간단히 데워 먹었어요 중복에 낙지랑 전복 넣고 끓이려고 구매했어요 간편해서 좋네요'

In [51]:
filtered_combined.loc[262, 'summarizing']

'삼계탕을 간편하게 데워서 여름철 보양식으로 즐기기 좋습니다. 조리 방법이 간단해 냉동된 제품을 끓는 물에 넣기만 하면 되며, 남은 국물로 닭죽을 만들어 먹는 것도 추천합니다. 아이와 함께 맛있게 먹을 수 있어 가족 단위로도 적합합니다. 중복이나 초복에 맞춰 다양한 재료를 추가해 변화를 주는 것도 좋은 아이디어입니다.'

In [52]:
filtered_combined.to_csv("처음한거.csv", index=False, encoding="utf-8-sig")

### `product_type`이 여러 개인 상품 다시 요약

- 전체 267개
- `product_type`이 여러 개인 상품 11개
- 위 상품들의 `product_type` 개수 27개
- 이 중 단어 수 10개 이상인 경우는 23개
- 따라서 총 267-11+23 = 279개의 리뷰 요약문이 있어야 함

In [116]:
grouped = df.groupby('product_id')['product_type'].nunique()
lst = grouped[grouped >= 2].index
lst

Int64Index([   5001235,    5035901,    5132102,    5133142,    5148360,
            1000048446, 1000085968, 1000175407, 1000323319, 1000323323,
            1000381876],
           dtype='int64', name='product_id')

In [117]:
cnt=0
for i in lst:
    print(df[df['product_id'] ==  i]['product_type'].unique(), "\n")
    cnt += df[df['product_id'] ==  i]['product_type'].nunique()
print(cnt)

['한우 양곰탕' '한우 곰탕' '한우 설렁탕'] 

['돼지 김치찌개' '참치 김치찌개'] 

['프리미엄 어묵탕 360g (시원한 맛)' '프리미엄 어묵탕 360g (얼큰한 맛)'] 

['육개장 500g' '미역국 500g' '소고기 장터국 500g'] 

['딱한끼 어묵탕 순한맛' '딱한끼 어묵탕 매운맛'] 

['한알만능육수 야채와 디포리 (3.7g x 22개입)' '한알만능육수 해물과 청양고추 (3.7g x 22개입)'] 

['등뼈 우거지 감자탕 900gX2개입' '뼈해장국 900gX2개입'] 

['양지 육수 300g 4개입' '멸치 육수 300g 4개입' '백합 육수 300g 4개입'] 

['The미식 차돌된장찌개 350g* 2개입' 'The미식 돼지고기김치찌개 350g* 2개입'
 'The미식 등심부대찌개 350g* 2개입'] 

['The미식 한우사태곰탕 400g * 2개입' 'The미식 양지육개장 350g * 2개입'
 'The미식 한우미역국 350g * 2개입'] 

['본토 마라탕 (진한맛)' '본토 마라탕 (순한맛)'] 

27


#### product_type이 1개인 상품 저장

In [118]:
df2 = pd.read_csv("처음한거.csv")
앞쪽 = df2[~df2['product_id'].isin(lst)]

print(앞쪽['product_id'].nunique())
앞쪽

256


,product_id,review,summarizing
1,5005071,추후 구매 후 참치 3 캔 기름기 빼서 넣었더니 아이들이 잘 먹네요 두부나 찌개 채...,참치 캔을 기름기 빼고 넣어 아이들과 함께 즐기기 좋습니다. 두부와 찌개 채소를 추...
2,5011633,어른처럼 소면 좋아하는 아이는 삶아서 넣어 먹으면 곰국 국수 질 좋은 다른 곰국도 ...,"소면을 삶아 곰국에 넣어 먹으면 맛있습니다. 아침에 밥을 말아 주기 좋고, 반찬이 ..."
3,5012851,리뷰가 좋아서 구매했어요 아침에 한 끼 국으로 좋았어요 청꼬 넣어서 푹 끓여서 한 ...,아침에 간편하게 국으로 즐기기 좋습니다. 청꼬와 함께 푹 끓여서 한 끼 해결할 수 ...
4,5012950,닭 계장할 때 베이스로 사골 국물 이용해서 쉽게 깊은 맛 냈어요 걍 찌개 베이스로 ...,사골 국물은 다양한 요리에 활용하기 좋습니다. 떡국이나 만두국을 만들 때 기본 육수...
5,5013198,도가니탕에 만두국 끓임 최고예요 입맛 없을 때 좋아요 간편해서 좋아요 더 다양한 상...,도가니탕에 만두를 추가해 끓이면 맛있는 만두국이 됩니다. 입맛이 없을 때 간편하게 ...
...,...,...,...
262,1000538774,삼계탕은 요걸로 정착입니다 삼계탕 마니아로써 조심스럽게 말씀 드리는데 올 여름 집에...,삼계탕을 간편하게 데워서 여름철 보양식으로 즐기기 좋습니다. 조리 방법이 간단해 냉...
263,1000605720,집에 있는 새우 더 넣고 쫄면 추가해서 짬뽕으로 먹었어요 다음엔 우동 면 추가해서 ...,집에 있는 새우와 쫄면을 추가해 짬뽕으로 즐기면 좋습니다. 다음에는 우동 면을 넣어...
264,1000605738,더워서 입맛이 없으시다면 소반 옥 육개장에 밥 말아서 한 그릇 뚝딱 해 보시는 걸 ...,"더운 날 입맛이 없을 때, 얼린 육개장을 냄비에 넣고 끓여서 밥과 함께 먹으면 간편..."
265,1000620989,배송을 받으시면 냉동고에 넣지 마시고 냉장고에 넣어서 하루 이상 냉장고에서 완전히 ...,"냉장고에서 하루 이상 해동한 후, 냄비에 옮겨 물을 약 1312ml 정도 넣고 끓이..."


In [119]:
df3 = df[~df['product_id'].isin(lst)][['product_id','product_type']].drop_duplicates()
df3.head(3)

,product_id,product_type
0,5013198,한우 도가니탕
75,1000306218,얼큰 대파 갈비탕
196,1000017002,소갈비탕


In [120]:
앞쪽 = 앞쪽.merge(df3, on='product_id', how='left')
new_column_order = ['product_id', 'product_type','review', 'summarizing']
앞쪽 =  앞쪽[new_column_order]
앞쪽

,product_id,product_type,review,summarizing
0,5005071,돼지고기 김치찌개,추후 구매 후 참치 3 캔 기름기 빼서 넣었더니 아이들이 잘 먹네요 두부나 찌개 채...,참치 캔을 기름기 빼고 넣어 아이들과 함께 즐기기 좋습니다. 두부와 찌개 채소를 추...
1,5011633,어린이 한우 사골 곰국,어른처럼 소면 좋아하는 아이는 삶아서 넣어 먹으면 곰국 국수 질 좋은 다른 곰국도 ...,"소면을 삶아 곰국에 넣어 먹으면 맛있습니다. 아침에 밥을 말아 주기 좋고, 반찬이 ..."
2,5012851,콩나물김칫국,리뷰가 좋아서 구매했어요 아침에 한 끼 국으로 좋았어요 청꼬 넣어서 푹 끓여서 한 ...,아침에 간편하게 국으로 즐기기 좋습니다. 청꼬와 함께 푹 끓여서 한 끼 해결할 수 ...
3,5012950,가족을 위한 사골곰탕,닭 계장할 때 베이스로 사골 국물 이용해서 쉽게 깊은 맛 냈어요 걍 찌개 베이스로 ...,사골 국물은 다양한 요리에 활용하기 좋습니다. 떡국이나 만두국을 만들 때 기본 육수...
4,5013198,한우 도가니탕,도가니탕에 만두국 끓임 최고예요 입맛 없을 때 좋아요 간편해서 좋아요 더 다양한 상...,도가니탕에 만두를 추가해 끓이면 맛있는 만두국이 됩니다. 입맛이 없을 때 간편하게 ...
...,...,...,...,...
251,1000538774,녹두삼계탕 900g,삼계탕은 요걸로 정착입니다 삼계탕 마니아로써 조심스럽게 말씀 드리는데 올 여름 집에...,삼계탕을 간편하게 데워서 여름철 보양식으로 즐기기 좋습니다. 조리 방법이 간단해 냉...
252,1000605720,짬뽕순두부찌개,집에 있는 새우 더 넣고 쫄면 추가해서 짬뽕으로 먹었어요 다음엔 우동 면 추가해서 ...,집에 있는 새우와 쫄면을 추가해 짬뽕으로 즐기면 좋습니다. 다음에는 우동 면을 넣어...
253,1000605738,듬뿍육개장,더워서 입맛이 없으시다면 소반 옥 육개장에 밥 말아서 한 그릇 뚝딱 해 보시는 걸 ...,"더운 날 입맛이 없을 때, 얼린 육개장을 냄비에 넣고 끓여서 밥과 함께 먹으면 간편..."
254,1000620989,토종닭 백숙 삼계탕,배송을 받으시면 냉동고에 넣지 마시고 냉장고에 넣어서 하루 이상 냉장고에서 완전히 ...,"냉장고에서 하루 이상 해동한 후, 냄비에 옮겨 물을 약 1312ml 정도 넣고 끓이..."


In [121]:
앞쪽.to_csv("앞쪽.csv", index=False, encoding="utf-8-sig")

#### product_type 2개 이상인거 재수집

In [122]:
뒤쪽 = df[df['product_id'].isin(lst)]
print(뒤쪽['product_id'].nunique())
뒤쪽

11


,product_id,product_type,review,category
842,5035901,돼지 김치찌개,냉동식품이라 보관도 편하고,보관
843,5035901,돼지 김치찌개,해동 후 끓여 주기만하면 되는 음식이라 조리도 굉장히 편해요,활용 방법
844,5035901,돼지 김치찌개,집에 양파 돼지고기 등 다른 재료 있으면 같이 넣고 끓이면 더 푸짐하고 맛있는 찌개...,활용 방법
845,5035901,돼지 김치찌개,국물이 맛있으니 면도 당연히 맛있었구요,맛/향/풍미
846,5035901,돼지 김치찌개,김치도 푹 익어서 부드럽고 고기가 정말 많이 들어 있었어요,양
...,...,...,...,...
22013,1000085968,등뼈 우거지 감자탕 900gX2개입,플라스틱이 깨져서 왔어요,포장/배송
22014,1000085968,등뼈 우거지 감자탕 900gX2개입,용기를 확인해 보니 저렇게 조각나서 깨져 있네요,포장/배송
22015,1000085968,등뼈 우거지 감자탕 900gX2개입,포장 과정에서 너무 던지나 봅니다,포장/배송
22016,1000085968,등뼈 우거지 감자탕 900gX2개입,끓이기 전에 용기 상태 꼭 확인하고 드세요,포장/배송


In [52]:
filtered_df = 뒤쪽[뒤쪽['category'] == '활용 방법']#.drop_duplicates()
filtered_df['product_id'].value_counts()

1000175407    115
5148360        32
5035901        22
5001235        22
5133142        21
1000048446     19
1000381876     13
5132102        12
1000323319     11
1000323323      9
1000085968      7
Name: product_id, dtype: int64

In [65]:
combined = filtered_df.groupby(['product_id', 'product_type'])['review'].agg(' '.join).reset_index()
filtered_combined = combined[combined['review'].apply(lambda x: len(x.split()) > 10)]

print(combined.shape[0], filtered_combined.shape[0])
filtered_combined = filtered_combined.reset_index(drop=True)
filtered_combined

27 23


,product_id,product_type,review
0,5001235,한우 곰탕,애정템 입니다 파 송송 당면 취향 것 소금 후추 톡톡 아이들 주려고 한우 곰탕도 주...
1,5001235,한우 양곰탕,집에 있는 새 송이버섯 애호박 대파 넣어 주고 계란 하나 푸니까 양 곰탕에다가 물을...
2,5035901,돼지 김치찌개,해동 후 끓여 주기만하면 되는 음식이라 조리도 굉장히 편해요 집에 양파 돼지고기 등...
3,5035901,참치 김치찌개,급히 김치찌개 땡길 때 참치 캔 하나 따 넣으면 괜찮은 듯합니다 햄이랑 두부 파 더...
4,5132102,프리미엄 어묵탕 360g (시원한 맛),김치 어묵 우동은 자주 시켜 먹는데 요것도 간식으로 주문해 봤어요 국물 밥 말아서 ...
5,5132102,프리미엄 어묵탕 360g (얼큰한 맛),먹기 간편해요 간단히 먹기에 좋지만 어묵탕 간단히 먹기 좋고 밤에 안주 혹은 야식으...
6,5133142,미역국 500g,물을 더 넣고 끓였습니다 간편하게 데워서 먹어요 해장할 때 미역국이 제일 좋더라구요...
7,5133142,소고기 장터국 500g,간편해서 좋아요 귀찮을 때 먹기 좋은 밀키트 한 끼 식사로 먹으려고 주문했어요
8,5133142,육개장 500g,식감과 시원한 맛을 위해 씻어 낸 숙주랑 대략 150200g 쯤 팽이버섯까지 넣고 ...
9,5148360,딱한끼 어묵탕 매운맛,간단하게 한 끼 해결이니 좋네요 조리도 어렵지 않아 좋네요 어묵탕 끓여서 한 끼 맛...


In [62]:
summarized = []

In [66]:
for i in range(filtered_combined.shape[0]):
    text = filtered_combined.loc[i, 'review']
    response = summarizing(text)
    summarized.append(response.content.strip('"'))

In [68]:
filtered_combined['summarizing'] = summarized
filtered_combined

,product_id,product_type,review,summarizing
0,5001235,한우 곰탕,애정템 입니다 파 송송 당면 취향 것 소금 후추 톡톡 아이들 주려고 한우 곰탕도 주...,"당면과 다양한 버섯, 배추, 야채를 넣어 전골로 끓여 먹으면 좋습니다. 양념을 만들..."
1,5001235,한우 양곰탕,집에 있는 새 송이버섯 애호박 대파 넣어 주고 계란 하나 푸니까 양 곰탕에다가 물을...,"집에 있는 새 송이버섯, 애호박, 대파를 넣고 계란을 추가한 후 양 곰탕에 물을 2..."
2,5035901,돼지 김치찌개,해동 후 끓여 주기만하면 되는 음식이라 조리도 굉장히 편해요 집에 양파 돼지고기 등...,해동 후 끓이기만 하면 간편하게 조리할 수 있습니다. 집에 있는 양파나 돼지고기 같...
3,5035901,참치 김치찌개,급히 김치찌개 땡길 때 참치 캔 하나 따 넣으면 괜찮은 듯합니다 햄이랑 두부 파 더...,"급하게 김치찌개가 생각날 때 참치 캔을 추가하면 좋습니다. 여기에 햄과 두부, 파를..."
4,5132102,프리미엄 어묵탕 360g (시원한 맛),김치 어묵 우동은 자주 시켜 먹는데 요것도 간식으로 주문해 봤어요 국물 밥 말아서 ...,김치 어묵 우동은 국물에 밥을 말아 한 끼로 즐기기 좋습니다. 출출할 때 간식으로 ...
5,5132102,프리미엄 어묵탕 360g (얼큰한 맛),먹기 간편해요 간단히 먹기에 좋지만 어묵탕 간단히 먹기 좋고 밤에 안주 혹은 야식으...,어묵탕으로 간편하게 즐기기 좋습니다. 간단히 조리할 수 있어 밤에 안주나 야식으로도...
6,5133142,미역국 500g,물을 더 넣고 끓였습니다 간편하게 데워서 먹어요 해장할 때 미역국이 제일 좋더라구요...,물을 더 넣고 끓여서 간편하게 데워 먹으면 좋습니다. 해장할 때 미역국이 특히 효과...
7,5133142,소고기 장터국 500g,간편해서 좋아요 귀찮을 때 먹기 좋은 밀키트 한 끼 식사로 먹으려고 주문했어요,간편하게 조리할 수 있어 바쁜 날에 적합한 밀키트입니다. 귀찮을 때 빠르게 한 끼를...
8,5133142,육개장 500g,식감과 시원한 맛을 위해 씻어 낸 숙주랑 대략 150200g 쯤 팽이버섯까지 넣고 ...,씻은 숙주와 팽이버섯을 넣고 12분 정도 끓이면 시원한 국물이 완성됩니다. 봉지째 ...
9,5148360,딱한끼 어묵탕 매운맛,간단하게 한 끼 해결이니 좋네요 조리도 어렵지 않아 좋네요 어묵탕 끓여서 한 끼 맛...,어묵탕으로 간편하게 한 끼를 해결할 수 있습니다. 냉동 꼬치 어묵을 추가하면 더욱 ...


In [69]:
filtered_combined.to_csv("뒤쪽.csv", index=False, encoding="utf-8-sig")

#### 전체 데이터 합치기

In [126]:
앞쪽 = pd.read_csv("앞쪽.csv")
앞쪽.head(4)

,product_id,product_type,review,summarizing
0,5005071,돼지고기 김치찌개,추후 구매 후 참치 3 캔 기름기 빼서 넣었더니 아이들이 잘 먹네요 두부나 찌개 채...,참치 캔을 기름기 빼고 넣어 아이들과 함께 즐기기 좋습니다. 두부와 찌개 채소를 추...
1,5011633,어린이 한우 사골 곰국,어른처럼 소면 좋아하는 아이는 삶아서 넣어 먹으면 곰국 국수 질 좋은 다른 곰국도 ...,"소면을 삶아 곰국에 넣어 먹으면 맛있습니다. 아침에 밥을 말아 주기 좋고, 반찬이 ..."
2,5012851,콩나물김칫국,리뷰가 좋아서 구매했어요 아침에 한 끼 국으로 좋았어요 청꼬 넣어서 푹 끓여서 한 ...,아침에 간편하게 국으로 즐기기 좋습니다. 청꼬와 함께 푹 끓여서 한 끼 해결할 수 ...
3,5012950,가족을 위한 사골곰탕,닭 계장할 때 베이스로 사골 국물 이용해서 쉽게 깊은 맛 냈어요 걍 찌개 베이스로 ...,사골 국물은 다양한 요리에 활용하기 좋습니다. 떡국이나 만두국을 만들 때 기본 육수...


In [125]:
뒤쪽 = pd.read_csv("뒤쪽.csv")
뒤쪽.head(4)

,product_id,product_type,review,summarizing
0,5001235,한우 곰탕,애정템 입니다 파 송송 당면 취향 것 소금 후추 톡톡 아이들 주려고 한우 곰탕도 주...,"당면과 다양한 버섯, 배추, 야채를 넣어 전골로 끓여 먹으면 좋습니다. 양념을 만들..."
1,5001235,한우 양곰탕,집에 있는 새 송이버섯 애호박 대파 넣어 주고 계란 하나 푸니까 양 곰탕에다가 물을...,"집에 있는 새 송이버섯, 애호박, 대파를 넣고 계란을 추가한 후 양 곰탕에 물을 2..."
2,5035901,돼지 김치찌개,해동 후 끓여 주기만하면 되는 음식이라 조리도 굉장히 편해요 집에 양파 돼지고기 등...,해동 후 끓이기만 하면 간편하게 조리할 수 있습니다. 집에 있는 양파나 돼지고기 같...
3,5035901,참치 김치찌개,급히 김치찌개 땡길 때 참치 캔 하나 따 넣으면 괜찮은 듯합니다 햄이랑 두부 파 더...,"급하게 김치찌개가 생각날 때 참치 캔을 추가하면 좋습니다. 여기에 햄과 두부, 파를..."


In [127]:
전체 = pd.concat([앞쪽, 뒤쪽]).reset_index(drop=True)
전체

,product_id,product_type,review,summarizing
0,5005071,돼지고기 김치찌개,추후 구매 후 참치 3 캔 기름기 빼서 넣었더니 아이들이 잘 먹네요 두부나 찌개 채...,참치 캔을 기름기 빼고 넣어 아이들과 함께 즐기기 좋습니다. 두부와 찌개 채소를 추...
1,5011633,어린이 한우 사골 곰국,어른처럼 소면 좋아하는 아이는 삶아서 넣어 먹으면 곰국 국수 질 좋은 다른 곰국도 ...,"소면을 삶아 곰국에 넣어 먹으면 맛있습니다. 아침에 밥을 말아 주기 좋고, 반찬이 ..."
2,5012851,콩나물김칫국,리뷰가 좋아서 구매했어요 아침에 한 끼 국으로 좋았어요 청꼬 넣어서 푹 끓여서 한 ...,아침에 간편하게 국으로 즐기기 좋습니다. 청꼬와 함께 푹 끓여서 한 끼 해결할 수 ...
3,5012950,가족을 위한 사골곰탕,닭 계장할 때 베이스로 사골 국물 이용해서 쉽게 깊은 맛 냈어요 걍 찌개 베이스로 ...,사골 국물은 다양한 요리에 활용하기 좋습니다. 떡국이나 만두국을 만들 때 기본 육수...
4,5013198,한우 도가니탕,도가니탕에 만두국 끓임 최고예요 입맛 없을 때 좋아요 간편해서 좋아요 더 다양한 상...,도가니탕에 만두를 추가해 끓이면 맛있는 만두국이 됩니다. 입맛이 없을 때 간편하게 ...
...,...,...,...,...
274,1000323319,The미식 차돌된장찌개 350g* 2개입,집에 쟁여 놓고 반찬 없을 때 먹어요 전 양지 육수랑 차돌 호박 두부 더 넣고 끓여...,"반찬이 없을 때 간편하게 활용할 수 있습니다. 양지 육수와 차돌, 호박, 두부를 추..."
275,1000323323,The미식 양지육개장 350g * 2개입,싱거운 거 좋아해서 냄비에 넣고 물 더 넣어서 끓이니까 딱 좋아요,싱거운 맛을 선호하는 분들은 냄비에 넣고 물을 추가하여 끓이면 적당한 간이 됩니다....
276,1000323323,The미식 한우사태곰탕 400g * 2개입,자취생이나 1 인 가구 분들께 한 끼 식사로 아주 좋을 거 같아요 완전 추천 입니다...,"겨울철에 간편하게 한 끼 식사로 즐기기 좋습니다. 자취생이나 1인 가구에 적합하며,..."
277,1000381876,본토 마라탕 (순한맛),숙주랑 버섯이 없기 때문에 숙주 버섯 따로 추가해서 먹으면 아이랑 같이 끓여 먹으니...,숙주와 버섯을 추가하여 아이와 함께 끓여 먹으면 재미있습니다. 더 순한 맛을 원한다...


In [128]:
전체['product_id'].nunique()

267

In [132]:
전체.to_csv("활용 방법 요약.csv", index=False, encoding="utf-8-sig")

#### 요약문 추출하는 함수

In [129]:
#pd.set_option('display.max_rows', None)
# 1000175407, 1000128834
전체[전체['product_id'] == 1000175407]

,product_id,product_type,review,summarizing
270,1000175407,멸치 육수 300g 4개입,육수 1개는 부족해 보이고 육수 2개는 있어야 1개를 끓일 수 있고 각종 찌개나 조...,"육수를 활용하여 다양한 요리를 간편하게 만들 수 있습니다. 예를 들어, 김치 수제비..."
271,1000175407,백합 육수 300g 4개입,더운 여름 따로 육수 낼 필요 없이 칼국수 생각이 간절했는데 육수를 직접 만들기는 ...,여름철에 육수를 따로 만들 필요 없이 간편하게 칼국수를 끓여 먹을 수 있습니다. 육...
272,1000175407,양지 육수 300g 4개입,바쁜 시간 간편하게 육수 낼 수 있어 정말 좋습니다 간단하게 요리할 수 있어서 참 ...,"육수를 간편하게 활용하여 다양한 요리를 즐길 수 있습니다. 예를 들어, 소고기 양지..."


In [130]:
def extract_summary(product_id):
    result = ""
    part = 전체[전체['product_id'] == product_id].reset_index(drop=True)
    num = part.shape[0]
    if num>1:
        for i in range(num):
            result += part.loc[i, 'product_type'] + ' : ' + part.loc[i, 'summarizing'] + "\n\n"
    else:
        result = part.loc[0, 'summarizing']
    return result

print(extract_summary(1000381876))

본토 마라탕 (순한맛) : 숙주와 버섯을 추가하여 아이와 함께 끓여 먹으면 재미있습니다. 더 순한 맛을 원한다면 채소를 더 넣거나 곰탕 육수를 추가해 보세요. 팽이버섯을 넣어도 좋고, 마라탕을 좋아하는 아이에게 내용물을 추가해 주면 든든한 한 끼가 됩니다.

본토 마라탕 (진한맛) : 숙주나물, 배추, 버섯, 고기 등을 추가하여 더욱 푸짐하게 즐길 수 있습니다. 배달 음식보다 훨씬 맛있고, 여러 가지 재료를 복잡하게 고를 필요 없이 간편하게 한 끼를 해결할 수 있습니다. 끓이는 방법도 라면처럼 간단해 누구나 쉽게 만들 수 있습니다.




In [133]:
df = pd.read_csv("활용 방법 요약.csv")

def extract_summary(product_id):
    result = ""
    part = df[df['product_id'] == product_id].reset_index(drop=True)
    num = part.shape[0]
    if num>1:
        for i in range(num):
            result += part.loc[i, 'product_type'] + ' : ' + part.loc[i, 'summarizing'] + "\n\n"
    else:
        result = part.loc[0, 'summarizing']
    return result

print(extract_summary(1000381876))

본토 마라탕 (순한맛) : 숙주와 버섯을 추가하여 아이와 함께 끓여 먹으면 재미있습니다. 더 순한 맛을 원한다면 채소를 더 넣거나 곰탕 육수를 추가해 보세요. 팽이버섯을 넣어도 좋고, 마라탕을 좋아하는 아이에게 내용물을 추가해 주면 든든한 한 끼가 됩니다.

본토 마라탕 (진한맛) : 숙주나물, 배추, 버섯, 고기 등을 추가하여 더욱 푸짐하게 즐길 수 있습니다. 배달 음식보다 훨씬 맛있고, 여러 가지 재료를 복잡하게 고를 필요 없이 간편하게 한 끼를 해결할 수 있습니다. 끓이는 방법도 라면처럼 간단해 누구나 쉽게 만들 수 있습니다.


